# Landscape Evolution Modeling

A showcase of xarray-simlab in the context of landscape evolution modeling (an almost real world example).

In [ ]:
import numpy as np
import xarray as xr
import xsimlab as xs

## Import and inspect a model

The model (i.e., the `xsimlab.Model` object) that we use here is provided by the [xarray-topo](https://gitext.gfz-potsdam.de/sec55-public/xarray-topo) package (**Note:** check the version of this package below, it may not correspond to the latest stable release).

In [ ]:
import xtopo
print(xtopo.__version__)

In [ ]:
from xtopo.models.fastscape_base import fastscape_base_model

This model simulates the long-term evolution of topographic surface elevation (hereafter noted $h$) on a 2D regular grid. The local rate of elevation change, $\partial h/\partial t$, is determined by the balance between uplift (uniform in space and time) $U$ and erosion $E$.

$$\frac{\partial h}{\partial t} = U - E$$

Total erosion $E$ is the combined effect of the erosion of (bedrock) river channels, noted $E_r$, and erosion- transport on hillslopes, noted $E_d$

$$E = E_r + E_d$$

Erosion of river channels is given by the stream power law:

$$E_r = K_r A^m (\nabla h)^n$$

where $A$ is the drainage area and $K$, $m$ and $n$ are parameters.

Erosion on hillslopes is given by a linear diffusion law:

$$E_d = K_d \nabla^2 h$$

We can see these parameters - as well as the initial elevation surface and the grid parameters - as model inputs in the `repr`.

In [ ]:
fastscape_base_model

To have a better picture of all processes (and inputs and/or variables) in the model, we can visualize it as a graph. Processes are in blue and inputs are in yellow. The order in the graph corresponds to the order in which the processes will be exectued during a simulation.

Note: the visualization requires graphviz and python-graphviz packages (both can be installed using conda and the conda-forge channel).

In [ ]:
fastscape_base_model.visualize(show_inputs=True)

More information can be shown for each process in the model, e.g., for the grid

In [ ]:
fastscape_base_model.grid

## Create a model setup

We create a simulation setup using the `create_setup` function.

In [ ]:
nx = 101
ny = 101

in_ds = xs.create_setup(
    model=fastscape_base_model,
    clocks={
        'time': np.linspace(0., 1e6, 101),
        'out': np.linspace(0., 1e6, 11)
    },
    master_clock='time',
    input_vars={
        'grid': {'x_size': nx, 'y_size': ny, 'x_length': 1e5, 'y_length' :1e5},
        'topography': {'elevation': (('y', 'x'), np.random.rand(ny, nx))},
        'flow_routing': {'pit_method': 'mst_linear'},
        'spower': {'k_coef': 7e-5, 'm_exp': 0.4, 'n_exp': 1},
        'diffusion': {'k_coef': 1.},
        'block_uplift': {'u_coef': 2e-3}
    },
    output_vars={
        'out': {'topography': 'elevation'},
        None: {'grid': ('x', 'y')}
    }
)

Some explanation about the arguments of `create_setup` and the values given above:

- we specify the model we want to use, here `fastscape_base_model`,
- we specify values for clock coordinates (i.e., time coordinates),
- among these coordinates, we specify the master clock, i.e., the coordinate that will be used to
  set the time steps,
- we set values for model inputs, grouped by process in the model,
- we set the model variables for which we want to take snapshots during a simulation, grouped first
  by clock coordinate (`None` means that only one snapshot will be taken at the end of the simulation)
  and then by process.
  
Here above, we define a 'time' coordinate and another coordinate 'out' with much larger but aligned
time steps (the values are in years). 'time' will be used for the simulation time steps and 'out' will be used to take just a few, evenly-spaced snapshots of
variable 'elevation' in process 'topography'. We also want to
save the $x$ and $y$ coordinates of the grid (values in meters), which are time-invariant.

The initial conditions consist here of a nearly flat topographic surface with small random perturbations.

`create_setup` returns a `xarray.Dataset` object that contains everything we need to run the simulation. 

In [ ]:
in_ds

If present, the metadata (e.g., description, units, math_symbol...) associated to each input variable in the model are added as attributes in the dataset, e.g.,

In [ ]:
in_ds.spower__k_coef

## Run the model

We run the model simply by calling `in_ds.xsimlab.run()`, which returns a new Dataset with both the inputs and the outputs. 

In [ ]:
out_ds = in_ds.xsimlab.run(model=fastscape_base_model)

out_ds

Note in `out_ds` the `topography__elevation` variable which has now an additional `out` dimension and also the new variables `grid__x` and `grid__y`.

## Analyse, plot and save the results

The simulation input and output data is already in a format that allows us using all the nice features of xarray to further analyse, process, plot and/or write to disk (e.g., in a netCDF file) the data.

In this case for example, before doing any further processing it is more convenient to set $x$ and $y$ coordinates as coordinates of the output `Dataset` instead of data variables, using the `set_index` method. We can easily chain this method with `xsimlab.run` as it both return Dataset objects: 

In [ ]:
out_ds = (in_ds
          .xsimlab.run(model=fastscape_base_model)
          .set_index(x='grid__x', y='grid__y'))

out_ds

It is then easier to plot the simulation outputs, e.g., here below the elevation values at the end of the simulation:

In [ ]:
%matplotlib inline

xr.plot.pcolormesh(out_ds.isel(out=-1).topography__elevation,
                   size=5, aspect=1);

xarray datasets can be used with [Holoview](http://holoviews.org/), a plotting package that is really helpful for quickly and interactively exploring multi-dimensional data.

In [ ]:
import holoviews as hv

hv.notebook_extension('bokeh')

We can for example see below how the relief is created during the simulation (snapshots are taken every 100000 years and elevation values are in meters).

In [ ]:
%%opts Image style(cmap='viridis') plot[colorbar=True]
%%opts Image [width=500, height=400]
hv_ds = hv.Dataset(out_ds.topography__elevation)
hv_ds.to(hv.Image, ['x', 'y'])

Additionally, We can compute derived quantities without much effort. Here below we calculate the surface denudation rates (in m/yr) averaged over each time steps of the output `out` dimension. 

In [ ]:
def denudation_rate(ds, time_dim='out'):
    topo = ds.topography__elevation
    dt = ds[time_dim].diff(time_dim)
    den_rate = topo.diff(time_dim) / dt - ds.block_uplift__u_coef
    return den_rate

In [ ]:
den_rate = denudation_rate(out_ds)

We further compute and plot the spatially averaged denudation rate.

In [ ]:
den_rate.mean(('x', 'y')).plot();

## Run the model with time-varying parameter values

Instead of providing constant, scalar values for model inputs, it is possible to provide arrays which have the same dimension as the one used for the "master clock" (the `time` dimension in this case).

As an example, we try below a sinusoidal variation for the $K$ parameter of the stream power law, with a period of 400000 years.

In [ ]:
da_k_time = 5e-5 + 3e-5 * np.cos((2 * np.pi / 4e5) * in_ds.time)

da_k_time.plot();

We then re-use the simulation setup created above and only update the parameters of the stream-power process with the new values for $K$ (using `Dataset.xsimlab.update_vars`).

Note the `time` dimension of the `spower__k_coef` variable in the new returned Dataset.

In [ ]:
in_ds_kt = in_ds.xsimlab.update_vars(
    model=fastscape_base_model,
    input_vars={'spower': {'k_coef': da_k_time, 'm_exp': 0.4, 'n_exp': 1}}
)

in_ds_kt

We then run the model, unstack the spatial dimensions, compute the denudation rates and plot the spatial averages, here again by easily chaining xarray and xarray-simlab methods on the input Dataset.

If we compare the results with those from the previous run, we clearly see the impact of the time-varying $K$ parameter values on the denudation rates. 

In [ ]:
den_rate_kt = (in_ds_kt
               .xsimlab.run(model=fastscape_base_model)
               .set_index(x='grid__x', y='grid__y')
               .pipe(denudation_rate))

den_rate_kt.mean(('x', 'y')).plot();

## Run and combine different model setups

Here is an brief example of running the model multiple times for different fixed values of $K$ and then concatenate the results into a single dataset. In next versions of xarray-simlab, this process will be even simpler.

In [ ]:
def run_model(k_value):
    print('run k=%f' % k_value)
    
    ivars = {'spower': {'k_coef': k_value, 'm_exp': 0.4, 'n_exp': 1}}
    
    out_ds = (in_ds
              .xsimlab.update_vars(model=fastscape_base_model,
                                   input_vars=ivars)
              .xsimlab.run(model=fastscape_base_model)
              .set_index(x='grid__x', y='grid__y'))
    
    return out_ds

 
out_ds_multi = xr.concat(
    [run_model(k) for k in (5e-5, 6e-5, 7e-5)],
    dim='spower__k_coef', data_vars='different'
)

Note the additional `spower__k_coef` dimension, which has its own coordinate with labels corresponding to the different $K$ values.

In [ ]:
out_ds_multi

This new dimension also appears in the Holoview figure

In [ ]:
%%opts Image style(cmap='viridis') plot[colorbar=True]
%%opts Image [width=500, height=400]
hv_ds = hv.Dataset(out_ds_multi.topography__elevation)
hv_ds.to(hv.Image, ['x', 'y'])

## Create an alternative version of the model

xarray-simlab makes it easy to create alternative versions of a model. In the example below, instead of using constant block uplift, we set a linear uplift function along the $x$ dimension. The first step is to create a new process, i.e., a Python class decorated by ``xsimlab.process``. 

In [ ]:
from xtopo.models.fastscape_base import Grid2D, ClosedBoundaryFaces


@xs.process
class VariableUplift(object):
    """Compute spatially variable uplift as a linear function of x."""
    
    x_coef = xs.variable(description='uplift function x coefficient')
    
    active_nodes = xs.foreign(ClosedBoundaryFaces, 'active_nodes')
    x = xs.foreign(Grid2D, 'x')
    
    uplift = xs.variable(intent='out', group='uplift')

    def initialize(self):
        mask = self.active_nodes
        ny, nx = mask.shape

        u_rate = np.ones((ny, nx)) * self.x_coef * self.x[None, :]
        
        self._u_rate = np.zeros((ny, nx))
        self._u_rate[mask] = u_rate[mask]

    def run_step(self, dt):
        self.uplift = self._u_rate * dt


We then update the model that we used above with the new process (note the change in repr: the `uplift` process has now an `x_coef` input).

In [ ]:
alt_model = (fastscape_base_model.drop_processes('block_uplift')
                                 .update_processes({'uplift_func': VariableUplift}))

alt_model

We then re-use our intial setup, remove from this setup everything that is not related to the new model (using `Dataset.xsimlab.filter_vars` which here drops the `uplift__u_coef` variable), update the setup with the new parameter and then run the model.

Note that in some cases it is convenient to use the `with` statement with a `Model` object. For example we don't need to provide the `model` argument in `filter_vars`, `update_vars` and `run` methods below. 

In [ ]:
with alt_model:
    out_ds_alt = (
        in_ds
        .xsimlab.filter_vars()
        .xsimlab.update_vars(input_vars={'uplift_func': {'x_coef': 1e-7}})
        .xsimlab.run()
        .set_index(x='grid__x', y='grid__y')
    )

You can compare the results obtained here with the results obtained above.

In [ ]:
xr.plot.pcolormesh(out_ds_alt.isel(out=-1).topography__elevation,
                   size=5, aspect=1);

In [ ]:
%%opts Image style(cmap='viridis') plot[colorbar=True]
%%opts Image [width=500, height=400]
hv_ds = hv.Dataset(out_ds_alt.topography__elevation)
hv_ds.to(hv.Image, ['x', 'y'])